In [21]:
import importlib
import os
import re
import sys
from collections import defaultdict, namedtuple
from dataclasses import dataclass, fields, asdict
from pathlib import Path
from mri_preproc.paths import hemond_data, init_paths
from mri_preproc import prepare_scans
import subprocess

Initialize paths (depends on what machine I'm running from)

In [22]:
init_paths.main()
from mri_preproc.paths.init_paths import DATA_HOME

Reload modules if I make any changes
- Double check that if I edit record.py, that reloading hd would reload those changes too, or if I need to use reload recursive

In [23]:
importlib.reload(hemond_data)
importlib.reload(prepare_scans)

<module 'mri_preproc.prepare_scans' from '/home/srs-9/Projects/ms_mri/mri_preproc/prepare_scans.py'>

In [24]:
dataset = hemond_data.collect_choroid_dataset(DATA_HOME, suppress_output=True)
dataset[0]

In [26]:
processing_script = "/home/srs-9/Projects/ms_mri/mri_preproc/process_scan.sh"
scan_dir = dataset[0].root
modality = "flair"
command_parts = [processing_script, str(scan_dir), modality]

CompletedProcess(args=['/home/srs-9/Projects/ms_mri/mri_preproc/process_scan.sh', '/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215', 'flair'], returncode=0)

In [19]:
try:
    result = subprocess.run(command_parts, capture_output=True, text=True, check=True)
except subprocess.CalledProcessError:
    pass

'/home/srs-9/Projects/ms_mri/mri_preproc/process_scan.sh /mnt/t/Data/3Tpioneer_bids/sub-1001/ses-20170215 flair'

## Examine 3Tpioneer_bids Data

In [9]:
dataroot = "/mnt/t/Data/3Tpioneer_bids"
dataset2 = hemond_data.scan_data_dir(dataroot)

/home/srs-9/Projects/ms_mri/mri_preproc/paths/hemond_data.py:112: UserWarning: No scan exists for subject: ms1492 and session: 20190620
  warnings.warn(


In [10]:
dataset2[0]

Scan(subid='ms1001', date='20170215', dataroot=PosixPath('/mnt/t/Data/MONAI/flair'), image=PosixPath('/mnt/t/Data/MONAI/flair/sub-ms1001_ses-20170215.nii.gz'), label=PosixPath('/mnt/t/Data/MONAI/flair/labels/sub-ms1001_ses-20170215.nii.gz'), cond=None)

In [9]:
subject = dataset[0]
proc_folder = subject.root / "proc"
if not proc_folder.is_dir(): 
    os.makedirs(proc_folder)

/mnt/t/Data/3Tpioneer_bids/sub-1001/ses-20170215/proc


In [9]:
for i in range(10):
    nii_image = prepare_scans.load_scan(dataset[i].image)
    print(nii_image.shape)

(240, 300, 300)
(239, 300, 300)
(210, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)


In [9]:
for i in range(10):
    nii_image = prepare_scans.load_scan(dataset[i].image)
    print(nii_image.shape)

(240, 300, 300)
(239, 300, 300)
(210, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)
(210, 300, 300)
(240, 300, 300)
(240, 300, 300)


In [6]:
training_data = []
test_data = []

for scan in dataset:
    if scan.cond == 'tr':
        training_data.append({"image": scan.image, "label": scan.label})
    elif scan.cond == 'ts':
        test_data.append(scan.image)